In [1]:
# Cell 1: install
!pip install --quiet streamlit opencv-python-headless pillow numpy pyngrok

# Optional: if you want nicer file-editing in the notebook
!pip install --quiet google-colab --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00


In [3]:
# Cell 2: write the streamlit app
app_code = r'''
import streamlit as st
from PIL import Image
import numpy as np
import cv2
import io
import sys
import traceback

st.set_page_config(page_title="Colab Streamlit Demo", layout="centered")

st.title("Streamlit frontend (works from Google Colab)")

st.sidebar.header("Options")
show_original = st.sidebar.checkbox("Show original image", value=True)
to_gray = st.sidebar.checkbox("Convert to grayscale", value=False)
to_edges = st.sidebar.checkbox("Canny edges", value=False)
edge_thresh1 = st.sidebar.slider("Canny thresh1", 10, 200, 50)
edge_thresh2 = st.sidebar.slider("Canny thresh2", 50, 300, 150)

uploaded = st.file_uploader("Upload an image (PNG / JPG)", type=["png","jpg","jpeg"])

# Simple built-in sample image
if uploaded is None:
    st.info("No image uploaded — using a sample image. (You can upload your own.)")
    # create a simple sample image
    img = np.zeros((320,480,3), dtype=np.uint8)
    cv2.putText(img, "Sample", (50,160), cv2.FONT_HERSHEY_SIMPLEX, 3, (255,200,100), 4, cv2.LINE_AA)
    image = Image.fromarray(img)
else:
    image = Image.open(uploaded).convert("RGB")

cols = st.columns(2)
if show_original:
    with cols[0]:
        st.subheader("Original")
        st.image(image, use_column_width=True)

# Convert to numpy BGR for OpenCV
img_np = np.array(image)[:, :, ::-1].copy()  # RGB->BGR

processed = img_np.copy()

if to_gray:
    processed = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    # display grayscale as PIL
    proc_pil = Image.fromarray(processed)
    with cols[1]:
        st.subheader("Grayscale")
        st.image(proc_pil, use_column_width=True)
    # if edges also requested, keep going
    if not to_edges:
        st.markdown("---")

if to_edges:
    if processed.ndim == 3:  # convert to gray if needed
        gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    else:
        gray = processed
    edges = cv2.Canny(gray, threshold1=edge_thresh1, threshold2=edge_thresh2)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    with cols[1]:
        st.subheader("Edges")
        st.image(Image.fromarray(edges_rgb), use_column_width=True)

# Simple download button for the processed image
def to_bytes(img_arr):
    # img_arr expected in RGB
    pil = Image.fromarray(img_arr[:, :, ::-1]) if img_arr.ndim == 3 else Image.fromarray(img_arr)
    buf = io.BytesIO()
    pil.save(buf, format="PNG")
    buf.seek(0)
    return buf

if st.button("Download processed (PNG)"):
    # choose appropriate output
    out = processed
    if out.ndim == 2:
        out_rgb = cv2.cvtColor(out, cv2.COLOR_GRAY2RGB)
    else:
        out_rgb = out
    st.download_button("Click to download", data=to_bytes(out_rgb), file_name="processed.png", mime="image/png")

st.markdown("---")
st.subheader("Quick code runner (limited)")

code_input = st.text_area("Run a tiny snippet that receives `image` as a PIL.Image and may return text or image (be careful!)",
                          value="from PIL import Image\n# image is available as 'image'\n'image size: %s' % str(image.size)")

if st.button("Run code"):
    # VERY restricted mini-runner (still dangerous to expose widely)
    local_env = {"image": image, "np": np, "cv2": cv2, "Image": Image}
    try:
        result = eval(compile(code_input, "<string>", "eval"), {}, local_env)
        if isinstance(result, Image.Image):
            st.image(result, caption="Result image", use_column_width=True)
        else:
            st.write(result)
    except Exception as e_eval:
        # fallback to exec if eval fails
        try:
            exec(compile(code_input, "<string>", "exec"), {}, local_env)
            st.success("Executed.")
        except Exception as e_exec:
            st.error("Error running code:")
            tb = traceback.format_exc()
            st.text(tb)
'''
with open('streamlit_app.py','w',encoding='utf-8') as f:
    f.write(app_code)

print("Wrote streamlit_app.py")


Wrote streamlit_app.py


In [4]:
# Cell 1: install (run once)
!pip install --quiet streamlit opencv-python-headless pillow numpy pyngrok


In [5]:
# Cell 2: write the streamlit app file (streamlit_app.py)
app_code = r'''
import streamlit as st
from PIL import Image
import numpy as np
import cv2
import io
import sys
import traceback

st.set_page_config(page_title="Colab Streamlit Demo", layout="centered")

st.title("Streamlit frontend (works from Google Colab)")

st.sidebar.header("Options")
show_original = st.sidebar.checkbox("Show original image", value=True)
to_gray = st.sidebar.checkbox("Convert to grayscale", value=False)
to_edges = st.sidebar.checkbox("Canny edges", value=False)
edge_thresh1 = st.sidebar.slider("Canny thresh1", 10, 200, 50)
edge_thresh2 = st.sidebar.slider("Canny thresh2", 50, 300, 150)

uploaded = st.file_uploader("Upload an image (PNG / JPG)", type=["png","jpg","jpeg"])

# Simple built-in sample image
if uploaded is None:
    st.info("No image uploaded — using a sample image. (You can upload your own.)")
    # create a simple sample image
    img = np.zeros((320,480,3), dtype=np.uint8)
    cv2.putText(img, "Sample", (50,160), cv2.FONT_HERSHEY_SIMPLEX, 3, (255,200,100), 4, cv2.LINE_AA)
    image = Image.fromarray(img)
else:
    image = Image.open(uploaded).convert("RGB")

cols = st.columns(2)
if show_original:
    with cols[0]:
        st.subheader("Original")
        st.image(image, use_column_width=True)

# Convert to numpy BGR for OpenCV
img_np = np.array(image)[:, :, ::-1].copy()  # RGB->BGR

processed = img_np.copy()

if to_gray:
    processed = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    # display grayscale as PIL
    proc_pil = Image.fromarray(processed)
    with cols[1]:
        st.subheader("Grayscale")
        st.image(proc_pil, use_column_width=True)
    # if edges also requested, keep going
    if not to_edges:
        st.markdown("---")

if to_edges:
    if processed.ndim == 3:  # convert to gray if needed
        gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    else:
        gray = processed
    edges = cv2.Canny(gray, threshold1=edge_thresh1, threshold2=edge_thresh2)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    with cols[1]:
        st.subheader("Edges")
        st.image(Image.fromarray(edges_rgb), use_column_width=True)

# Simple download button for the processed image
def to_bytes(img_arr):
    # img_arr expected in RGB
    pil = Image.fromarray(img_arr[:, :, ::-1]) if img_arr.ndim == 3 else Image.fromarray(img_arr)
    buf = io.BytesIO()
    pil.save(buf, format="PNG")
    buf.seek(0)
    return buf

if st.button("Download processed (PNG)"):
    # choose appropriate output
    out = processed
    if out.ndim == 2:
        out_rgb = cv2.cvtColor(out, cv2.COLOR_GRAY2RGB)
    else:
        out_rgb = out
    st.download_button("Click to download", data=to_bytes(out_rgb), file_name="processed.png", mime="image/png")

st.markdown("---")
st.subheader("Quick code runner (limited)")

code_input = st.text_area("Run a tiny snippet that receives `image` as a PIL.Image and may return text or image (be careful!)",
                          value="from PIL import Image\n# image is available as 'image'\n'image size: %s' % str(image.size)")

if st.button("Run code"):
    # VERY restricted mini-runner (still dangerous to expose widely)
    local_env = {"image": image, "np": np, "cv2": cv2, "Image": Image}
    try:
        result = eval(compile(code_input, "<string>", "eval"), {}, local_env)
        if isinstance(result, Image.Image):
            st.image(result, caption="Result image", use_column_width=True)
        else:
            st.write(result)
    except Exception as e_eval:
        # fallback to exec if eval fails
        try:
            exec(compile(code_input, "<string>", "exec"), {}, local_env)
            st.success("Executed.")
        except Exception as e_exec:
            st.error("Error running code:")
            tb = traceback.format_exc()
            st.text(tb)
'''
with open('streamlit_app.py','w',encoding='utf-8') as f:
    f.write(app_code)

print("Wrote streamlit_app.py")


Wrote streamlit_app.py


In [9]:
# Cell 3: start Streamlit + ngrok and print the URL
import os, time, subprocess, threading
from pyngrok import ngrok

def in_colab():
    try:
        import google.colab
        return True
    except Exception:
        return False

# If you have an ngrok authtoken, you can set it here (uncomment and set your token)
ngrok.set_auth_token("318b9IITbqiWfdoqVpjTMf22afg_5uXjKqdvf79vegEwCUSaf") # <--- Replace "YOUR_NGROK_AUTHTOKEN" with your actual ngrok authtoken

if in_colab():
    print("Detected Google Colab — creating ngrok tunnel for port 8501...")
    public_url = ngrok.connect(8501).public_url
    print("Public URL (open this in your browser):", public_url)
    cmd = ["streamlit", "run", "streamlit_app.py", "--server.port=8501", "--server.headless=true", "--server.enableCORS=false"]
else:
    print("Not running in Colab — open http://localhost:8501 after this starts.")
    cmd = ["streamlit", "run", "streamlit_app.py", "--server.port=8501"]

# Start streamlit process
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=os.getcwd())

# Stream logs to notebook output (non-blocking)
def stream_logs(pipe):
    for line in iter(pipe.readline, b''):
        try:
            print(line.decode('utf-8', errors='ignore'), end='')
        except:
            pass

threading.Thread(target=stream_logs, args=(proc.stdout,), daemon=True).start()

time.sleep(2)
print("Streamlit launched (pid: {})".format(proc.pid))
if in_colab():
    print("Open the public URL above to view the app.")
else:
    print("Open http://localhost:8501 in your browser.")

Detected Google Colab — creating ngrok tunnel for port 8501...
Public URL (open this in your browser): https://3f26a0fb1da7.ngrok-free.app
2025-08-11 11:01:34.248 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.173.54.236:8501

Streamlit launched (pid: 3474)
Open the public URL above to view the app.
